## Многоагентная реализация разговорного агента

Мы видели, что в примере выше Graph RAG хорошо работает в том случае, если во входном текста находятся какие-то именованные сущности. Если же таких сущностей нет, то непонятно, от какой начальной точки отталкиваться.

В итоге, для формирования ответа можно предложить такой подход:

* Ищем именованные сущности
* Если они найдены, и соответствуют сущностям в графе - применяем Graph RAG
* Если сущностей нет - используем классический RAG

![](img/step1.svg)

Для реализации такой логики удобно использовать **многоагентный подход**. В этом случае:

"Каждый шаг пайплайна (NER, Graph RAG, Simple RAG) оформляется в виде отдельного **агента**. В большинстве случаев агент действует на основе LLM.
"Каждый агент получает на вход состояние. Внутри состояния может храниться общая для всех агентов история диалога, или же информация о текущем состоянии решения задачи может передаваться в другом виде.
"После запуска агент маршрутизирует выполнение и передает его следующему агенту.

Для начала опишем набор фраз, на которых мы будем экспериментировать:

In [13]:
sentences = [
    "Расскажи всё, что ты знаешь про сорт вина мерло",
    "Какой виноград используют для изготовления красного вина?",
    "Стоит ли пить шампанское с утра?",
    "Какие вина подойдут к итальянской пасте?",
    "Какие вина едят с рыбой?"
]

Состояние задачи будем представлять словарём, со следующими полями:
* `input` - начальный запрос пользователя
* `entity` - сюда будем помещать список распознанных сущностей
* `output` - сюда в конечном итоге попадёт ответ системы

Для формирования начального состояния опишем функцию:

In [2]:
def mkstate(text):
    return { 'input' : text }

Для начала загрузим данные для Graph RAG: 

In [3]:
import json

with open('../graph_rag/graphs/entities.json',encoding='utf-8') as f:
    entities = json.load(f)
with open('../graph_rag/graphs/relations.json',encoding='utf-8') as f:
    relations = json.load(f)

Для порядка будем наследовать всех агентов от базового класса. Вызов агента принимает на вход состояние задачи (`state`) и возвращает название узла пайплайна, куда надо перейти дальше (`namestate`):

In [6]:
class Agent:
    
    def __call__(self, state):
        # do something
        return "new_namestate"


Исходя из этого, опишем агента для извлечения сущностей. Для создания экземпляра агента нам нужно будет передать ему промпт, пару имен состояний - для успешного завершения и для неуспеха, и список допустимых сущностей:

In [16]:
from yandex_chain import YandexLLM, YandexGPTModel
import os

auth = {
    "folder_id" : os.environ['folder_id'].strip(), 
    "api_key" : os.environ['api_key'].strip()
}

class NER(Agent):
    def __init__(self, prompt, on_success, on_fail, labels=None):
        self.llm = YandexLLM(model=YandexGPTModel.Pro, **auth)
        self.prompt = prompt
        self.labels = labels
        self.lowlabels = [ x.lower() for x in labels ] if labels else None
        self.on_success = on_success
        self.on_fail = on_fail

    def __call__(self, state):
        res = self.llm.invoke(self.prompt.replace('{}',state['input']))
        if res=="NONE" or res=='(NONE)':
            return self.on_fail
        if '('in res: res = res[res.index('(')+1:]
        if ')' in res: res = res[:res.index(')')]
        res = [x.strip() for x in res.split('|')]
        if self.labels:
            res = [ x for x in res if x.lower() in self.lowlabels ]
        res = [x for x in res if x!="NONE"]
        if len(res)==0:
            return self.on_fail
        state['entities'] = res
        return self.on_success

Теперь опишем агента для извлечения сущностей из запроса:

In [18]:
GeneralNER = NER(
"""
В запросе приводится короткий текст. Тебе необходимо выделить из него все сущности следующих типов: название вина, сорт винограда, название блюда. Верни только список сущностей, присутствующих в тексте, в скобках через знак |, например: (сира|ЮАР). Верни только те сущности, которые в явном виде присутствуют в запросе. Не придумывай никакие дополнительные сущности и не рассуждай. Если сущностей в тексте нет, верни NONE. Включать NONE в список не надо. В списке не должно быть никаких сущностей, которых нет в тексте.
--- текст ---
```{}```
""",
"graph_rag","simple_rag",
labels = entities.keys()
)

for s in sentences:
    x = GeneralNER(mkstate(s))
    print(f"{s} -> {x}")

Расскажи всё, что ты знаешь про сорт вина мерло -> graph_rag
Какой виноград используют для изготовления красного вина? -> graph_rag
Стоит ли пить шампанское с утра? -> graph_rag
Какие вина подойдут к итальянской пасте? -> simple_rag
Какие вина едят с рыбой? -> simple_rag


Реализуем агента для Graph RAG. Его логика уже была раскрыта нами ранее (см. папку graph_rag).

In [19]:
import networkx as nx

class GraphRAG(Agent):

    answer_prompt = """
Тебе задан следующий запрос от пользователя: {question}.
Ответь на этот вопрос, используя при этом информацию, содержащуюся ниже в тройных обратных кавычках:
```
{context}
```
"""

    def __init__(self,entities,relations,level=2):
        self.llm = YandexLLM(**auth)
        self.entities = entities
        self.relations = relations
        self.level = level

    def populate_graph(self,G,e,level=None):
        if e in G.nodes:
            return
        if e in self.entities.keys():
            G.add_node(e, label=e)
        if level is not None and level<=0:
            return
        new_ent = set(
            [r['source'] for r in relations if r['target'] == e] + 
            [r['target'] for r in relations if r['source'] == e])
        for ne in new_ent:
            self.populate_graph(G,ne,None if level is None else level-1)
        for r in relations:
            if r['source'] == e:
                G.add_edge(e, r['target'], label=r['relation'], desc=r['desc'])
            if r['target'] == e:
                G.add_edge(r['source'], e, label=r['relation'], desc=r['desc'])

    def __call__(self, state):
        G = nx.DiGraph()
        for x in state['entities']:
            self.populate_graph(G,x,self.level)
        ctx = '\n'.join(e[-1]['desc'] for e in G.edges(data=True))
        state['output'] = self.llm.invoke(self.answer_prompt
            .replace('{context}',ctx)
            .replace('{question}',state['input']))
        return 'конец'
    
TheGraphRAG = GraphRAG(entities,relations,2)


Аналогичным образом опишем агента для наивного RAG:

In [25]:
from langchain_chroma import Chroma
from yandex_chain import YandexEmbeddings
import langchain.chains
import langchain.prompts
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

class NaiveRAG(Agent):

    prompt = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Выведи только
краткий ответ, не надо пояснительного текста.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

    def join_docs(self,docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def __init__(self,num_documents=5):
        self.embeddings = YandexEmbeddings(**auth)
        self.db = Chroma(embedding_function=self.embeddings, persist_directory='../simple_rag/chroma_db')
        self.llm = YandexLLM(model=YandexGPTModel.Pro,**auth)
        self.template = langchain.prompts.PromptTemplate(
            template=self.prompt, input_variables=["context", "question"])
        self.retriever = self.db.as_retriever(
            search_type="mmr", search_kwargs={"k": num_documents})
        self.chain = (
            {"context": self.retriever | self.join_docs, "question": RunnablePassthrough()}
            | self.template
            | self.llm
            | StrOutputParser()
        )

    def __call__(self, state):
        state['output'] = self.chain.invoke(state['input'])
        return 'конец'

TheNaiveRAG = NaiveRAG()

Для оркестрации вызовов агентов создадим класс `AgentRuntime`. Ему мы явно передадим соответствие имён состояний и агентов, которых нужно вызывать в этих состояниях:

In [26]:
class AgentRuntime:
    def __init__(self,states):
        self.states = states
        
    def run(self, state, start_state, verbose=False):
        s = start_state
        while True:
            if verbose:
                print(f"Executing state: {s}, state = {state}")
            A = self.states[s]
            s = A(state)
            if s == 'конец' or s is None:
                break
        return state

table = {
    'начало' : GeneralNER,
    'simple_rag' : TheNaiveRAG,
    'graph_rag' : TheGraphRAG
}

AR = AgentRuntime(table)
AR.run(mkstate(sentences[0]), 'начало', verbose=True)

Executing state: начало, state = {'input': 'Расскажи всё, что ты знаешь про сорт вина мерло'}
Executing state: graph_rag, state = {'input': 'Расскажи всё, что ты знаешь про сорт вина мерло', 'entities': ['мерло']}


{'input': 'Расскажи всё, что ты знаешь про сорт вина мерло',
 'entities': ['мерло'],
 'output': '**Сорт винограда мерло** — это красный сорт винограда, который широко выращивается в разных странах мира. Вот некоторые интересные факты о нём:\n\n* **Происхождение:** точно неизвестно, где впервые появился сорт мерло, но он имеет французские корни. Впервые о мерло упомянули в регионе Либурне во Франции.\n\n*  **Распространение:** мерло выращивают в США, Италии, Франции и других странах. В США он популярен в штатах Вашингтон, Нью-Йорк и Калифорния. Также этот сорт культивируют в Окленде, который расположен в Новой Зеландии вокруг Окленда, на Корсике (Франция) и в некоторых других регионах мира. \n\n * **Характеристики винограда:** виноград мерло обычно даёт небольшие круглые ягоды глубокого сине-чёрного цвета с лёгким восковым налётом. Вкус ягод считается более мягким, чем у некоторых других сортов.\n\n **Вино из мерло:**\n\n- Вино, произведённое из винограда сорта мерло, характеризуется гл

Проверим, как это работает для всех наших предложений:

In [27]:
for s in sentences:
    print('-'*30)
    print(f"Вопрос: {s}")
    state = mkstate(s)
    AR.run(state, 'начало', verbose=True)
    print(f"Результат: {state['output']}")

------------------------------
Вопрос: Расскажи всё, что ты знаешь про сорт вина мерло
Executing state: начало, state = {'input': 'Расскажи всё, что ты знаешь про сорт вина мерло'}
Executing state: graph_rag, state = {'input': 'Расскажи всё, что ты знаешь про сорт вина мерло', 'entities': ['мерло']}
Результат: **Описание сорта винограда Мерло:**

* **Происхождение:** сорт винограда Мерло происходит из Франции. Впервые о нём упомянули в регионе Либурн.

* **Цвет:** виноград Мерло даёт красные вина разнообразных оттенков, цвет которых ассоциируется с оперением птицы чёрный дрозд.

**Распространение:**

 * Виноград Мерло выращивают не только во Франции, но и в других винодельческих регионах мира, включая США, Италию и Новую Зеландию. Так, в США посадки винограда Мерло можно найти в штатах Вашингтон, Вирджиния, Нью-Йорк и Калифорния. В Италии его выращивают в регионах Венето, Эмилия-Романья, Тоскана, Лацио, Калабрия, Базиликата и Сардиния. А в Новой Зеландии — в Окленде. 

 **Особенности с

## Более сложный пример

Одной из часто встречающихся задач является подбор вина к еде и наоборот. Поскольку это достаточно формализованный процесс, существуют описанные экспертами **онтологии**, описывающие эту задачу. В файлах [Food.rdf](../source/food.rdf) и [Wine.rdf](../source/wine.rdf) содержатся соответствующие онтологии, а файл [OntologyReco.ipynb](OntologyReco.ipynb) содержит эксперименты с этими онтологиями и пример запуска логического вывода для подбора вина к еде и наоборот.

В этом случае наша многоагентная система будет состоять из следующих агентов:
* Входной классификатор `InputClassifier`, который будет определять тип нашего запроса - подбор вина к еде, подбор еды к вину или другой запрос.
* Извлечение сущностей `FoodNER`, которая будет извлекать из текста названия блюд. В случае, если классификатор посчитал, что запрос на подбор вина, а NER не нашел названия блюда - мы откатываемся к состоянию *другой запрос*.
* Если сущности нашлись - используем агент `WineFinder` для поиска вина в соответствии с онтологией
* Поиск блюда к вину осуществляется аналогичным образом, мы это не будет тут рассматривать.
* В состоянии *другой запрос* мы повторяем логику агента выше, и используем Simple RAG или Graph RAG.

В итоге получаем такую диаграмму межагентного взаимодействия:

![](img/step2.svg)

Для начала реализуем входной классификатор. Его для простоты реализуем на основе Zero-Shot подхода с помощью Yandex GPT Classfier API.

> В качестве упражнения рекомендуем попробовать улучшить классификатор, добавив к нему примеров. Это делается через параметр `samples` в конструкторе классификатора.

In [36]:
from yandex_chain import YandexGPTClassifier

class Classfier(Agent):
    def __init__(self, task_description, labels, samples=None):
        self.classifier = YandexGPTClassifier(task_description, labels, samples, **auth)

    def __call__(self, state):
        res = self.classifier.invoke(state['input'])
        c = self.classifier.get_top_label(res)
        return c
    
InputClassifier = Classfier(
    """Определи, содержится ли в вопросе одна из следующих задач:
* подобрать вино к еде (подбор_вина),
* подобрать еду к вину (подбор_еды),
* другой вопрос (другая_тема)""",
    ["подбор_вина","подбор_еды","другая_тема"]
)

for s in sentences:
    res = InputClassifier(mkstate(s))
    print(f"{s} -> {res}")

Расскажи всё, что ты знаешь про сорт вина мерло -> другая_тема
Какой виноград используют для изготовления красного вина? -> подбор_вина
Стоит ли пить шампанское с утра? -> другая_тема
Какие вина подойдут к итальянской пасте? -> подбор_вина
Какие вина едят с рыбой? -> подбор_вина


Теперь реализуем извлекатор различных блюд, упомянутых в нашей онтологии. Для простоты опишем все блюда и их русскоязычное описание в отдельном словаре, а затем уже этот словать в форматированном виде добавим в промпт.

In [37]:
Food = {
"BlandFish" : "пресная рыба",
"CheeseNutsDessert" : "сырный десерт (возможно, с орехами)",
"DarkMeatFowl" : "тёмное мясо птицы",
"Dessert" : "десерт",
"Fish" : "рыба",
"Fruit" : "фрукты",
"LightMeatFowl" : "лёгкое блюдо из мяса птицы",
"OtherTomatoBasedFood" : "блюдо из томатов",
"OysterShellfish" : "устрицы",
"PastaWithHeavyCream" : "паста со сливками",
"PastaWithLightCream" : "лёгкая паста со сливками",
"PastaWithNonSpicyRedSauce" : "паста с неострым красным соусом",
"PastaWithSpicyRedSauce" : "паста с острым красным соусом",
"RedMeat" : "красное мясо",
"Seafood" : "морепродукты",
"Shellfish" : "моллюски",
"SpicyRedMeat" : "острое красное мясо",
"SweetDessert" : "сладкий десерт",
"SweetFruit" : "сладкие фрукты"
}

descr = "\n".join([f" * {k} - {v}" for k,v in Food.items()])

FoodNER = NER(
    """Ниже в тройных обратных кавычках приводится текст. Твоя задача - выделить из этого текста название упомянутых там блюд, из приведённого ниже списка:
{descr}
В качестве результата выведи только список сущностей из упомянутого списка, в круглых скобках через знак |, например (RedMeat|SpicyRedMeat). Если в запросе не содержится упоминания ни об одной из сущностей, выведи NONE. Не выводи ничего кроме результирующего списка или слова NONE.
--- текст ---
```{}```
""".replace("{descr}",descr),
"найти_вино","другая_тема",
Food.keys()
)

for s in sentences:
    x = mkstate(s)
    res = FoodNER(x)
    print(f"{s} -> {res} {x.get('entities','')}")

Расскажи всё, что ты знаешь про сорт вина мерло -> другая_тема 
Какой виноград используют для изготовления красного вина? -> другая_тема 
Стоит ли пить шампанское с утра? -> другая_тема 
Какие вина подойдут к итальянской пасте? -> другая_тема 
Какие вина едят с рыбой? -> найти_вино ['Fish']


Теперь опишем агента по поиску подходящего вина:

In [53]:
import pytholog as pl

class WineFinder(Agent):
    def __init__(self):
        self.kb = pl.KnowledgeBase('foodmatch')  
        self.program = [ "match(F,W) :- foodmatch(F,hasColor,C), wine(W,hasColor,C), foodmatch(F,hasSugar,S), wine(W,hasSugar,S), foodmatch(F,hasFlavor,U), wine(W,hasFlavor,U), foodmatch(F,hasBody,B), wine(W,hasBody,B)"]
        self.kb([ x.strip()[:-1] for x in open('foodmatch.pl').readlines() if x[0]!='#' ]+self.program)      

    def __call__(self, state):
        res = set()
        for f in state['entities']:
            q = self.kb.query(pl.Expr(f"match(food_{f},W)"))
            if q==['No']:
                continue
            res |= { list(x.values())[0] for x in q }
        wines = ", ".join([ x[5:] for x in res ])
        state['output'] = f"Вам подойдут следующие вина:\n{wines}"

WineFinderAgent = WineFinder()

Теперь опишем таблицу меж-агентных переходов и запустим всю многоагентную систему:

In [54]:
table = {
    'начало' : InputClassifier,
    'подбор_вина' : FoodNER,
    'найти_вино' : WineFinderAgent,
    'другая_тема' : GeneralNER,
    'simple_rag' : TheNaiveRAG,
    'graph_rag' : TheGraphRAG
}

AR = AgentRuntime(table)
AR.run(mkstate(sentences[-1]), 'начало', verbose=True)

Executing state: начало, state = {'input': 'Какие вина едят с рыбой?'}
Executing state: подбор_вина, state = {'input': 'Какие вина едят с рыбой?'}
Executing state: найти_вино, state = {'input': 'Какие вина едят с рыбой?', 'entities': ['Fish']}


{'input': 'Какие вина едят с рыбой?',
 'entities': ['Fish'],
 'output': 'Вам подойдут следующие вина:\nWhiteWine, Semillon, TableWine, SauvignonBlanc, DryWine, WhiteTableWine, WhiteBordeaux, Tours, WhiteNonSweetWine, PinotBlanc, Sancerre, SemillonOrSauvignonBlanc, EarlyHarvest, DryRiesling, CheninBlanc, WhiteLoire, WhiteBurgundy, Meursault, Riesling, DryWhiteWine, Chardonnay, Burgundy'}

## Дальнейшие улучшения

В данном примере мы не рассмотрели обратную задачу подбору вина - подбору блюда под конкретное вино. Это чуть более сложная задача из-за большого количества конкретных вин в онтологии. Также было бы логично подбирать как блюда, так и сорта вина по отдельным параметрам (цвет, сладость, тело, вкус).

Существенного улучшения качества диалога можно добиться, добавив в многоагентную систему проверку на адекватность ответа. В этом случае в конце диалога мы ставим LLM, которая оценивает, насколько ответ системы хорошо подходит под изначальный вопрос пользователя. Если совпадение неполное, то вопрос перефразируется LLM-кой, и затем цикл подбора ответа на вопрос повторяется. Схема такой обратной петли показана на рисунке ниже:

![](img/step3.svg)